In [ ]:
import numpy as np
import sys
from numpy import linalg as LA
import math
from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation
plt.style.use('seaborn-pastel')

In [ ]:
γ = 5/3
T = 0.02
L = 10
vL, vR = 0, 0
PL, PR = 1013250, 101325
ρL, ρR = 13, 1.3

In [ ]:
def c(e):
    c = np.sqrt(γ*(γ-1)*e)
    return c


def Ω(u,c):
    A = np.array(([-u*c,c,γ-1],[-c**2,0,γ-1],[u*c,-c,γ-1]))
    return A


def Ωinv(A):
    return LA.inv(A)


def Λ(u,c):
    A = np.array(([u+c,0,0],[0,u,0],[0,0,u-c]))
    return A


def MatMultiply(A,B,C):
    return (A.dot(B)).dot(C)

1)Задания начальных данных

In [ ]:
t = 0.0
τ = 5e-5
NX = 101
CFL = 0.01
P = np.zeros(NX)
e = np.zeros(NX)
ρ = np.zeros(NX)
u = np.zeros(NX)
P1 = np.zeros(NX)
e1 = np.zeros(NX)
ρ1 = np.zeros(NX)
u1 = np.zeros(NX)
Pdata = np.zeros((66,NX))
udata = np.zeros((66,NX))
ρdata = np.zeros((66,NX))
for i in range(NX):
    P[i] = (PR,PL)[i<50]
    ρ[i] = (ρR,ρL)[i<50]
    u[i] = (vR,vL)[i<50]
    e[i] = P[i]/((γ-1) * ρ[i])
Pdata[0][:] = P
udata[0][:] = u
ρdata[0][:] = ρ

2)Генерация сетки

In [ ]:
h = 2*L/(NX - 1)
X = np.linspace(-L,L, NX)

Схема КИР:

In [ ]:
def CIR(x0,x1,x2,A,B,τ):
    x = x1 -(τ/(2*h)) * A.dot(x2-x0) + (τ/(2*h)) * B.dot(x2 - 2 * x1 + x0)
    return x

In [ ]:
k = 0
while t < T:
    for i in range(1,NX-1):
    ## Расчитываем матрицы
        ω = Ω(u[i],c(e[i]))
        ωinv = Ωinv(ω)
        λ = Λ(u[i],c(e[i]))
        λabs = np.abs(λ)
        A = MatMultiply(ωinv,λ,ω)
        B = MatMultiply(ωinv,λabs,ω)
    ## Проверяем выполнения условия устойчивости
        if (τ*max(np.diagonal(λabs))/h) > CFL :
            τ = h/(max(np.diagonal(λabs) * 100.0))
        if τ < 1e-7:
            τ = 1e-7
    ## Расчет вектора в консервативных переменных
        w0 = np.array([ρ[i-1],ρ[i-1]*u[i-1],ρ[i-1]*e[i-1]])
        w1 = np.array([ρ[i],ρ[i]*u[i],ρ[i]*e[i]])
        w2 = np.array([ρ[i+1],ρ[i+1]*u[i+1],ρ[i+1]*e[i+1]])
        
        w = CIR(w0,w1,w2,A,B,τ)
        ρ1[i] = w[0]
        u1[i] = w[1] / w[0]
        e1[i] = w[2] / w[0]
        P1[i] = (γ - 1.0) * ρ1[i] * e1[i]     
    ## Учет граничных условий
    ρ1[0] = ρ1[1]
    u1[0] = u1[1]
    e1[0] = e1[1]
    P1[0] = P1[1]

    ρ1[NX - 1] = ρ1[NX - 2]
    u1[NX - 1] = u1[NX - 2]
    e1[NX - 1] = e1[NX - 2]
    P1[NX - 1] = P1[NX - 2]
    ## Запись результатов(каждые 100 итераций)               
    if(k % 100 == 0):
        udata[int(k/100)][:] = u1
        ρdata[int(k/100)][:] = ρ1
        Pdata[int(k/100)][:] = P1
    ρ = ρ1
    u = u1
    e = e1
    P = P1
    ## Инкрементирование
    t += τ
    k += 1

In [ ]:
%matplotlib notebook
plt.style.use('seaborn-pastel')

fig = plt.figure()
ax = plt.axes(xlim=(-10, 10), ylim=(0, 2e6))
ax.set_ylabel("P,Па")
ax.set_xlabel("x,м")
plt.title('График P(t)') 
plt.grid()
line, = ax.plot([], [], lw=3)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

def init():
    time_text.set_text('')
    line.set_data([], [])
    return line,
def animate(i):
    x = X
    y = Pdata[i]
    time_text.set_text(i)
    line.set_data(x, y)
    return line,
 
anim = FuncAnimation(fig, animate, init_func=init,
                               frames=65, interval=300, blit=True)
anim.save('gif/график p(t).gif', writer='imagemagick')

![Alt Text](https://i.imgur.com/yoJdT03.gif)

In [ ]:
%matplotlib notebook
plt.style.use('seaborn-pastel')

fig = plt.figure()
ax = plt.axes(xlim=(-10, 10), ylim=(-10, 500))
ax.set_ylabel("u,м/с")
ax.set_xlabel("x,м")
plt.title('График u(t)') 
plt.grid()
line, = ax.plot([], [], lw=3)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

def init():
    time_text.set_text('')
    line.set_data([], [])
    return line,
def animate(i):
    x = X
    y = udata[i]
    time_text.set_text(i)
    line.set_data(x, y)
    return line,
 
anim = FuncAnimation(fig, animate, init_func=init,
                               frames=65, interval=300, blit=True)
anim.save('gif/график u(t).gif', writer='imagemagick')

![Alt Text](https://i.imgur.com/7oJKQCf.gif)

In [ ]:
%matplotlib notebook
plt.style.use('seaborn-pastel')

fig = plt.figure()
ax = plt.axes(xlim=(-10, 10), ylim=(0, 20))
ax.set_ylabel("ρ,кг/м^3")
ax.set_xlabel("x,м")
plt.title('График ρ(t)') 
plt.grid()
line, = ax.plot([], [], lw=3)
time_text = ax.text(0.02, 0.95, '', transform=ax.transAxes)

def init():
    time_text.set_text('')
    line.set_data([], [])
    return line,
def animate(i):
    x = X
    y = ρdata[i]
    time_text.set_text(i)
    line.set_data(x, y)
    return line,
 
anim = FuncAnimation(fig, animate, init_func=init,
                               frames=65, interval=300, blit=True)
anim.save('gif/график ρ(t).gif', writer='imagemagick')

![Alt Text](https://i.imgur.com/yfM0U0i.gif)